In [1]:
import math
import torch
import torch.nn as nn

c:\Users\Asus\anaconda3\envs\rl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Network Architecture

In [2]:

# Initial part of network, whihc is fixed
class initial_part_network(nn.Module):
    def __init__(self):
        super(initial_part_network, self).__init__()
        self.conv = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5, stride=1, padding=2)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv(x)
        x = self.maxpool(x)
        return x

# Custom part of network, which can be added many time as per requirement(0,1,2,3,...times)
class custom_part(nn.Module):
    def __init__(self, in_channels):
        super(custom_part, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=2 * in_channels, kernel_size=7, stride=1, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2 * in_channels, out_channels=4 * in_channels, kernel_size=5, stride=1, padding=2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        return x

# Final network is which formed with intial part network first then attached custom blocks as per given condition
class final_network(nn.Module):
    def __init__(self, num_custom_block=1):
        super(final_network, self).__init__()
        self.initial_part = initial_part_network()
        
        # Stacking custom blocks based on given num custom block value
        custom_blocks = []
        in_channels = 8
        for _ in range(num_custom_block):
            custom_blocks.append(custom_part(in_channels))
            in_channels *= 4  
        self.custom_blocks = nn.Sequential(*custom_blocks)
        
        # Last part of network
        self.remaining_network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_channels, 3)  # 3 is arbitrary number, it modify as per requirement
        )

    def forward(self, x):
        x = self.initial_part(x)
        x = self.custom_blocks(x)
        x = self.remaining_network(x)
        return x


Model (for demonstration, usinng 3 custom blocks)

In [3]:
model = final_network(num_custom_block=3)  
print(model)


final_network(
  (initial_part): initial_part_network(
    (conv): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (custom_blocks): Sequential(
    (0): custom_part(
      (conv1): Conv2d(8, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): custom_part(
      (conv1): Conv2d(32, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False

Receptive Field Calculation

In [4]:
# Calculating output-input parameters
def in_out_parameter(layer_parameters, layer_input):
    h_in, w_in, j_in, r_in = layer_input
    k, s, p = layer_parameters

    # Calculate output dimensions for height and width
    h_out = math.floor((h_in - k + 2 * p) / s) + 1
    w_out = math.floor((w_in - k + 2 * p) / s) + 1

    # Output jump and receptive field calculation
    j_out = j_in * s
    r_out = r_in + (k - 1) * (j_in)
  
    return (h_out, w_out, j_out, r_out)


In [5]:
# Function to print layer information
def layer_information(layer, layer_name):
    print(f"{layer_name}:")
    print(f"\t n (H, W): ({layer[0]}, {layer[1]})")
    print(f"\t jump: {layer[2]}")
    print(f"\t Receptive fied: {layer[3]}")


In [6]:
# Calculation of receptive field
def receptive_field(model, input_size):
    result = []

    # input img parameters
    current_layer = [input_size[0], input_size[1], 1, 1]  
    # Print corresponding details
    layer_information(current_layer,"Input image")

    for layer_name, layer in model.named_modules():
        # 
        if isinstance(layer, (nn.Conv2d, nn.MaxPool2d)):
            # Extracting kernal
            k = layer.kernel_size[0] if isinstance(layer.kernel_size, tuple) else layer.kernel_size
            # Strides
            s = layer.stride[0] if isinstance(layer.stride, tuple) else layer.stride
            # padding
            p = layer.padding[0] if isinstance(layer.padding, tuple) else layer.padding

            # Calculatinng receptive field for next layer
            parameters = (k, s, p)
            current_layer = in_out_parameter(parameters, current_layer)

            result.append(current_layer)
            layer_information(current_layer, layer_name)

    return result


In [7]:
# Final model and input img size
model = final_network(num_custom_block=2)
input_image_size = (320, 960)  


In [8]:
# Calculate receptive fields
receptive_field(model, input_image_size)

Input image:
	 n (H, W): (320, 960)
	 jump: 1
	 Receptive fied: 1
initial_part.conv:
	 n (H, W): (320, 960)
	 jump: 1
	 Receptive fied: 5
initial_part.maxpool:
	 n (H, W): (160, 480)
	 jump: 2
	 Receptive fied: 6
custom_blocks.0.conv1:
	 n (H, W): (160, 480)
	 jump: 2
	 Receptive fied: 18
custom_blocks.0.maxpool1:
	 n (H, W): (80, 240)
	 jump: 4
	 Receptive fied: 20
custom_blocks.0.conv2:
	 n (H, W): (80, 240)
	 jump: 4
	 Receptive fied: 36
custom_blocks.0.maxpool2:
	 n (H, W): (40, 120)
	 jump: 8
	 Receptive fied: 40
custom_blocks.1.conv1:
	 n (H, W): (40, 120)
	 jump: 8
	 Receptive fied: 88
custom_blocks.1.maxpool1:
	 n (H, W): (20, 60)
	 jump: 16
	 Receptive fied: 96
custom_blocks.1.conv2:
	 n (H, W): (20, 60)
	 jump: 16
	 Receptive fied: 160
custom_blocks.1.maxpool2:
	 n (H, W): (10, 30)
	 jump: 32
	 Receptive fied: 176


[(320, 960, 1, 5),
 (160, 480, 2, 6),
 (160, 480, 2, 18),
 (80, 240, 4, 20),
 (80, 240, 4, 36),
 (40, 120, 8, 40),
 (40, 120, 8, 88),
 (20, 60, 16, 96),
 (20, 60, 16, 160),
 (10, 30, 32, 176)]

Number of parameters calculation

In [9]:
def calculate_parameters(custom_block):

    total_parameter = 0

    for layer in custom_block.children():

        if isinstance(layer, nn.Conv2d):

            # No. of weights = out_channels*in_channels*kernel_height*kernel_width
            weights = layer.out_channels * layer.in_channels * layer.kernel_size[0] * layer.kernel_size[1]

            # No. of bias is based on output channels
            biases = layer.out_channels

            # Final
            layer_parameter = weights + biases
            print(f"{layer}: Weights={weights}, Bias={biases}, Total={layer_parameter}")
            total_parameter += layer_parameter
            
    return total_parameter

In [10]:
# Calculatinng for 2nd custom block
second_custom_block = model.custom_blocks[1]  

In [11]:
second_block_parameter = calculate_parameters(second_custom_block)
print(f"\nNumber of learnable parameters in the second custom block : {second_block_parameter}")

Conv2d(32, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3)): Weights=100352, Bias=64, Total=100416
Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)): Weights=204800, Bias=128, Total=204928

Number of learnable parameters in the second custom block : 305344
